<a href="https://colab.research.google.com/github/MWaser/BurstIQ/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is for the purposes of
### 1.   Familiarizing the reader with BurstIQ's admin tool cliq2
### 2.   Allowing the developer to create a new interactive API for all BurstIQ operations from the cliq2 source code



# 1. Upgrade Python Version if necessary

The BurstIQ API requires Python 3.11 or greater.  Check the version to see if upgrading is necessary.

In [1]:
!python --version

Python 3.10.12


If upgrading is necessary, run the next box.
Do not worry about the virtual machine warnings -- a notebook is already a virtual machine.

In [2]:
!sudo apt-get update -y
!sudo apt-get install python3.11

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2

!python --version

!sudo apt-get install python3-pip

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,617 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [50.4 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,309 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,340 kB]
Hit:13 https://ppa.launchpadcontent.n

# 2. Install BurstIQ Command Line (cliq2)

Grab Python Wheel Installation file from Github Repo

In [3]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/cliq2-1.0.0-py3-none-any.whl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21896  100 21896    0     0  93135      0 --:--:-- --:--:-- --:--:-- 93174


Install from Wheel file

In [4]:
!pip3 install cliq2-1.0.0-py3-none-any.whl

Processing ./cliq2-1.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 KB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 KB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 KB 15.9 MB/s eta 0:00:00
  Created wheel for clint: filename=clint-0.5.1-py3-none-any.whl size=34473 sha256=a4c1a2f8bf551008b3365f0b1ed193ef63d51bb98893b8f13451367

Verify that command line program is installed correctly

In [5]:
!cliq2 -h

usage: cliq2 [-h] [-p PROPS_FILE]
             {version,drop_customer,drop_sdz,update_customer,predict,update_dict,import_dbschema,export_dbschema,truncate_chain,load,query,create_smart,execute_smart}
             ...

Command Line IQ (CLIQ) working with LifeGraph (aka GraphChain): Performs many of the admin-centric
commands necessary to support/manage a customer and its SDZs.

positional arguments:
  {version,drop_customer,drop_sdz,update_customer,predict,update_dict,import_dbschema,export_dbschema,truncate_chain,load,query,create_smart,execute_smart}
                        type "<cmd> --help" for help on specific command
    version             version number
    drop_customer       drop a customer by short name, BIQ_ADMIN only
    drop_sdz            drop a customer's sdz by short name, BIQ_ADMIN only
    update_customer     upserts a customer JSON configuration file, BIQ_ADMIN only
    predict             uses a data file to predict the dictionary and mapping
    update_dict      

In [6]:
!cliq2 update_customer -h

usage: cliq2 update_customer [-h] --file FILE

options:
  -h, --help            show this help message and exit
  --file FILE, -f FILE  the file path for the input


# 3. Build the Config file (cliq2.yml)

Documentation is at https://admin.burstiq.com/docs/pages/metadata_tools/cliq2/cliq2_desc.html#configuration

Example cliq2.yml:
> biq_admin:
>>  username: burstiq.admin  
>>  password: XXXXXXXXXXXXXXXXXX  
>>  server: https://keycloak.app.burstiq.com/auth  
>>  client_id: aaaaaaaaaaaa  
>>  realm: bbbbbbbbbbbb

> sdz_admin:
>>  username: john.wayne  
>>  password: XXXXXXXXXXXXXXXXXX  
>>  server: https://keycloak.app.burstiq.com/auth  
>>  client_id: burst  
>>  realm: { customer_shortname }  

> graphchain:
>>  server: https://app.burstiq.com  
>>  customer: { customer_shortname }  
>>  sdz: { customer_sdz_name }  

Use secrets for username, password, customer_shortname and customer_sdz name (the key on the left menu) to generate cliq.yml <br />
<b>IMPORTANT!!! </b> Be sure to delete the results of the following step before saving or all your secret information will be in the save file!

In [ ]:
from google.colab import userdata
yml = open(r"cliq2.yml", "w")
yml.write("sdz_admin:\n");
yml.writelines("   username: " + userdata.get('username') + "\n");
yml.writelines("   password: " + userdata.get('password') + "\n");
yml.writelines("   server: https://keycloak.app.burstiq.com/auth\n");
yml.writelines("   client_id: burst\n");
yml.writelines("   realm: " + userdata.get('customer_shortname') + "\n");
yml.writelines("graphchain:\n");
yml.writelines("   server: https://app.burstiq.com\n");
yml.writelines("   customer: " + userdata.get('customer_shortname') + "\n");
yml.writelines("   sdz: " + userdata.get('customer_sdz_name') + "\n");
yml.close();

!cat cliq2.yml


# 4. Initialize the new API

Copy Cliq2 Wheel file and unzip original BurstIQ source files from it

In [8]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/cliq2-1.0.0-py3-none-any.whl

! cp cliq2-1.0.0-py3-none-any.whl API.zip
! unzip -o API.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21896  100 21896    0     0  79912      0 --:--:-- --:--:-- --:--:-- 80205
Archive:  API.zip
  inflating: __init__.py             
  inflating: cliq2.py                
  inflating: cliq_properties.py      
  inflating: consts.py               
  inflating: util.py                 
  inflating: actions/__init__.py     
  inflating: actions/cliq_action.py  
  inflating: actions/create_smart_contract.py  
  inflating: actions/drop_customer.py  
  inflating: actions/drop_sdz.py     
  inflating: actions/execute_smart_contract.py  
  inflating: actions/export_dbschema.py  
  inflating: actions/import_dbschema.py  
  inflating: actions/load_data.py    
  inflating: actions/predict.py      
  inflating: actions/query.py        
  inflating: actions/truncate_chain.py  
  inflating: actions/update_customer.py  
  inflating: actions/up

Import and validate CliqProperties

In [9]:
import sys
sys.path.append('/usr/local/lib/python3.11/dist-packages/')
print("IGNORE clint SYNTAX warning (if clint has not been imported before)")
import clint
from cliq_properties import CliqProperties
from util import Util
props = CliqProperties(Util.standardize_file("cliq2.yml"))
print(props._graphchain)

IGNORE clint SYNTAX warning (if clint has not been imported before)
{'server': 'https://app.burstiq.com', 'customer': 'clinical_squared', 'sdz': 'dev1'}


/usr/local/lib/python3.11/dist-packages/clint/textui/prompt.py:68: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if prompt[-1] is not ' ':


Install and validate upgraded /graphchain_client/graphchain_client.py

In [13]:
!curl -H 'Accept: application/vnd.github.v3.raw' -O \
  -L https://api.github.com/repos/clinical-squared/BurstIQ/contents/graphchain_client/graphchain_client.py
!mv graphchain_client.py graphchain_client
from actions.cliq_action import CliqAction
print ("CliqAction imported!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7915  100  7915    0     0  42702      0 --:--:-- --:--:-- --:--:-- 42553
CliqAction imported!


# 5. Start Working the API

Grab an access token for future API calls

In [21]:
cp = CliqProperties("cliq2.yml")
realm = userdata.get('customer_shortname')
un = userdata.get('username')
pw = userdata.get('password')
token = cp._get_token("https://keycloak.app.burstiq.com/auth", "burst", realm, un, pw)
print(token)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ3MmdSOXFZaGpJVWZCNS00S3NSY2w2ODVsVUkzaExHcHdsa3d2dXRuQUVnIn0.eyJleHAiOjE3MDUzNTU0NDksImlhdCI6MTcwNTM1NDI0OSwianRpIjoiYzU2NjQ3NGMtZGI5MC00MDJiLWEyZjgtODI1MzUwZTU0MTYwIiwiaXNzIjoiaHR0cHM6Ly9rZXljbG9hay5hcHAuYnVyc3RpcS5jb20vYXV0aC9yZWFsbXMvY2xpbmljYWxfc3F1YXJlZCIsImF1ZCI6ImRldjEiLCJzdWIiOiI1OTFhZDA2ZC01OTQ1LTRhYmEtYjUzNy03MGM2OTQ5M2QzYWQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJidXJzdCIsInNlc3Npb25fc3RhdGUiOiJmYzU5YjYzYy1hYWRlLTQ0NjQtYTJjYS04YTU3MmNhMWQyMzciLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vYWRtaW4uYnVyc3RpcS5jb20iXSwicmVzb3VyY2VfYWNjZXNzIjp7ImRldjEiOnsicm9sZXMiOlsic2R6VXNlciIsInNkekFkbWluIl19fSwic2NvcGUiOiJlbWFpbCBwcm9maWxlIiwic2lkIjoiZmM1OWI2M2MtYWFkZS00NDY0LWEyY2EtOGE1NzJjYTFkMjM3IiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJuYW1lIjoiV2FycmVuIFBlbHR6IiwicHJlZmVycmVkX3VzZXJuYW1lIjoid2FycmVuLnBlbHR6IiwiZ2l2ZW5fbmFtZSI6IldhcnJlbiIsImZhbWlseV9uYW1lIjoiUGVsdHoiLCJlbWFpbCI6IndhcnJlbi5wZWx0ekBjbGluaWNhbHNxdWFyZWQuY29tIn0.iT-EC4YbMHBAB6aRaqsUKbwK